# [CogSci 131] Assignment 4
**Keilyn Yuzuki**
Due: February 27, 2020 at 11am

[+10 extra points] if you make it choose uniformly at random between all of the available matching rules

Ensure that at least 5 rules have one variable and at least 5 have two variables. 

1. Turn in your code [20pts, HELP]. (If you are going for bonus points, write a sentence on how you accomplished it)

2. Turn in two conversations [10pts each] of about 1-2 pages each: first, you talking to the chat bot about the domain, showing how well it responds. Second, give it to a friend who is told nothing about it other than to talk to them about this topic. Be sure that your conversations make clear whether it is a human or ELIZA who is speaking on each line. Write down some of your friend's impressions.

3. Based on how your ELIZA performs in conversations, describe in a sentence or two one way you could improve its performance, in principle, other than adding more rules [10pts, SOLO].

4. Finally, turn in a single sentence that you think nobody’s implementation will perform well on, but every human would. Explain why [5pts, SOLO].

In [1]:
# dependencies
import numpy as np
import pandas as pd
import csv
import re

# The Questions
There are 17 different types of questions that my ELIZA implementation can answer about Lilo and Stitch, along with a handful of generic questions. 12 rules have 1 variable and 5 of have 2 variables, and the generic rules look for exact string matches. The basic interaction allows for answering the following questions/statements:
- **hello**: responds with a random greeting from [*'Hi', 'Hello!', 'Hello', 'Howdy', 'Hey', 'Salutations', 'Hola'*]
- **who are you**: responds with "*I'm Eliza, and I'm an expert on Lilo and Stitch*"
- **how are you**: responds with a random selection from [*I'm fine, I'm good, TIRED, I'm a computer, I don't have feelings., I'm doing alright, How are you?, No really, how are you.*]
- **how do you know so much about lilo and stitch**: responds with a random selection from [*I just do., I was programmed to., I do Disney trivia on weekends, M A G I C*]
- It also has some basic programming to respond to input such as "I am fine", "I'm fine, or asking a question back to the user when asked about itself. If the user ever types in *yes*, ELIZA will respond with "*I see*"

### One Variable Rules
- 1.**who is ___**: Takes in one argument, which should be the name of a character. Responds with a description of that character from the `characters` dictionary, or from the `exp` table if the name of the character is given as `experiment #`.
- 2.**what is experiment ___**: Takes in one argument, which should be a number. Responds with a description of that experiment from the `exp` table.
- 3.**what is ___**: Answers questions about experiments and characters when referred to by name.
- 4.**what does experiment ___ do**: Answers what each experiment does when referred to by number. Argument should be a number corresponding to an experiment in the `exp` table.
- 5.**what does ___ do**: Answers what different characters do, whether a human character or an experiment listed in the `exp` table.
- 6.**what is ____, where ____ contains 'your' and 'favorite'**: A similar rule to the **what is ___** rule, this one is more specific in that it also checks for the words 'your' and 'favorite', and answers with what its favorite experiment is.
- 7.**favorite is ___**: Responds to statements about teh user's favorite character, when referred to by name.
- 8.**favorite experiment is ___**
- 9.**favorite is experiment __**: Rules 8 and 9 are defined in the same function, and respond to the user's favorite experiment.
- 10.**where does ___ live**: Responds to questions about where a character lives
- 11.**what is ___ job**: Responds to questions about the job a character has
- 12.**how tall is ___**: Responds to questions about the height of each character

### Two Variable Rules
- 1. **season ____ episode ____**: Answers questions about what happened in a given season and episode of the Lilo and Stitch TV Series
- 2. **____ is better than ____**: Responds to statements asserting that one thing is better than another
- 3. **love ____ and ____**: Responds to a statement about loving two things
- 4. **does ____ travel to ____** and **does ____ go to _____**: Responds to statements about characters travelling or going to places.
- 5. **can ____ ____** and **is it possible for ____ to ____**: Two rules that are meant to answer questions about whether a character has a certain ability.

# The Knowledge
The following describe the types of knowledge that the bot has and uses to respond to questions with

- `greetings`: a list of greetings for the bot to respond with when greeted

- `non_answers`: a list of frames for answering questions that don't match a rule

- `exp`: a dataframe containing information on the numbers, names, and descriptions of the experiments that Jumba created. Information was taken from [Fandom.com](https://liloandstitchfanfiction.fandom.com/wiki/List_of_Experiments)

- `ep`: a dataframe containing summaries of every episode of Lilo & Stitch: The Series. Information was taken from [Lilo & Stitch: The Series Episodes Descriptions on Wikipedia](https://en.wikipedia.org/wiki/List_of_Lilo_%26_Stitch:_The_Series_episodes)

- `characters`: a dictionary containing information about the main characters in the movies. Information was adapted from [List of Lilo & Stitch Characters on Wikipedia](https://en.wikipedia.org/wiki/List_of_Lilo_%26_Stitch_characters)

- `houses`: a dictionary containing information about where some of the main characters live. Information is what I remember from watching the Lilo and Stitch movies.

- `jobs`: a dictionary containing information about the jobs/occupations of some of the main characters. Information is what I remember from watching the Lilo and Stitch movies.

- `locations`: a dictionary containing information about where each character travels to. Information is based on what I remember from watching the movies.

- `heights`: a dictionary containing information about the height of each character. Lilo's height was found through Google on the [Lilo and Stitch Fandom Page](https://liloandstitch.fandom.com/wiki/Lilo_Pelekai), and the rest were extrapolated from the image below. The image is from the [List of Lilo & Stitch Characters Wikipedia page](https://en.wikipedia.org/wiki/List_of_Lilo_%26_Stitch_characters)

<img src="LiloStitchCharacters.jpg">

In [2]:
greetings = np.array(['Hi', 'Hello!', 'Hello', 'Howdy', 'Hey', 'Salutations', 'Hola'])
non_answers = np.array(['What are your opinions about ', 'What do you think about ', 'How do you feel about ',
                       "I'm sorry, I don't understand "])


exp = pd.read_csv('data/experiments.csv')
exp = exp[~(exp['Num'].isna()) & ~(exp['Name'].isna()) & ~(exp['Short Description'].isna())]
exp['Short Description'] = exp['Short Description'].astype(str)
exp['name_lower'] = exp['Name'].str.lower()

ep = pd.read_csv('data/all-episodes.csv')

characters = {'lilo':"Lilo Pelekai is one of the main characters, who lives with her older sister Nani",
            'stitch':"Stitch is Experiment 626, created by Jumba",
            'nani':"Nani is Lilo's older sister who takes care of her",
            'jumba':"Jumba is an evil genius who created Stitch and his cousins",
            'david':"David is Nani's coworkers, who later becomes her boyfriend",
            'scrump':"Scrump is Lilo's rag doll, which she made herself",
            'pleakley':"Agent Wendell Pleakley is Jumba's partner, and is an expert on Planet Earth",
            'gantu':"Gantu is a captain of the Galactic Federation",
            'mertle':"Mertle is Lilo's ex-best friend",
            'cobra':"Cobra Bubbles is a social worker who must evaluate Nani as Lilo's guardian",
            'grand councilwoman':'The Grand Councilwoman is the leader of the United Galactic Federation'}
houses = {'lilo':'Lilo lives in a house outside of Kokaua Town, Hawaii',
        'stitch':'Stitch lives with Lilo and Nani in their house outside of Kokaua Town, Hawaii',
        'jumba':'Jumba lives with Lilo, Nani, Stitch, and Pleakley in Kokaua Town, Hawaii',
        'pleakley':'Pleakley lives with Lilo, Nani, Stitch, and Jumba in Kokaua Town, Hawaii',
        'gantu':'Gantu lives in his spaceship next to a waterfall in Hawaii',
         'reuben':'Reuben lives with Gantu in his spaceship'}
jobs = {'lilos':"Lilo doesn't really have a job. She just explores with Stich",
       'stitchs':"Stitch doesn't really have a job. He is very mischevious and causes trouble.",
       'nanis':'Nani does her best to find work, despite Lilo and Stitch wreaking havoc everywhere',
       'pleakleys':"Pleakley is the Planet Earth Expert in the Galactic Federation",
       'jumbas':'Jumba prefers to be called an Evil Genius',
       'gantus':'Gantu is mostly responsible for capturing experiments',
       'davids':'David is a good surfer, but I am not sure what he does to earn money'}
locations = {'lilo':['the beach', 'the store', 'space', 'the volcano', 'home', 'the mall', 'school', 'hula practice', 'the luau'],
          'stitch':['the beach', 'the store', 'space', 'volcano', 'home', 'mall', 'animal shelter', 'the luau'],
          'nani':['work', 'home','the beach', 'hotel', 'market', 'store', 'restaurant', 'the luau'],
           'david':['the beach', 'nanis house', 'lilos house', 'store', 'restaurant', 'the luau'],
           'cobra':['lilos house', 'nanis house', 'the lighthouse'],
          'jumba':['the beach', 'space', 'jail', 'earth'],
          'pleakley':['the beach', 'space', 'the forest', 'earth'],
          'gantu':['earth', 'space', 'the waterfall']}
heights = {'lilo':'Lilo is 3 feet 5 inches tall',
          'stitch':'Stitch is around 3 feet tall',
          'nani':'If Lilo is ~3.5 feet tall, Nani must be at least 8 feet tall when looking at them next to eachother.',
           'david':'If Lilo is ~3.5 feet tall, David must be at least 9 feet tall when looking at them next to eachother.',
           'cobra':'If Lilo is ~3.5 feet tall, Cobra Bubbles must be at least 10 feet tall when looking at them next to eachother.',
          'jumba':'Jumba is shorter than Nani, but is probably around 7 feet tall',
          'pleakley':'Pleakley is shorter than Jumba, so is probably around 6 feet tall with his antenna',
          'gantu':'Gantu is around 20 feet tall'}

In [3]:
exp.sample(n=5)

,Num,Pod Color,Name,Short Description,Episode,name_lower
251,251,Green,Link,A small yellow Grundo-like experiment with red...,212,link
328,328,NaN,Bugle,Designed to distract the enemy by playing the ...,Leroy & Stitch,bugle
439,439,NaN,Winceslaws,Designed to turn people into mice.,Leroy & Stitch,winceslaws
445,445,NaN,No-C,Designed to take your CDs and transport them t...,Leroy & Stitch,no-c
166,166,NaN,Peppah,Designed to put pepper on 155's burger. If you...,Leroy & Stitch,peppah


In [4]:
ep.sample(n=5)

,season,episode,description
63,2,25,When Lilo and Stitch learn from Nosy that all ...
36,1,37,"Lilo's baseball team loses to Mertle's team, d..."
2,1,3,"After causing chaos at a beauty salon, Lilo an..."
19,1,20,Lilo and Stitch visit a planetarium and overhe...
51,2,13,"Keoni is going away for the weekend, so Lilo u..."


# Functions for Basic Interaction

In [5]:
def hello(s):
    """Responds to greetings from the user"""
    if s in [x.lower() for x in greetings]: 
        print('ELIZA: ', np.random.choice(greetings))
        print()
        return 1
    return 0
            

In [6]:
def common_questions(s):
    """Responds to common questions"""
    s_words = s.split()
    response = "It's good to hear you are"
    
    if s == 'who are you':
        print("I'm Eliza, and I'm an expert on Lilo and Stitch")
        print()
        return 1
    
    if s == 'how are you':
        r = np.random.choice(["I'm fine", "I'm good", "TIRED", "I'm a computer, I don't have feelings.", 
                                 "I'm doing alright", "How are you?", "No really, how are you."])
        print(r)
        print()
        return 1
    
    if s == 'how do you know so much about lilo and stitch':
        r = np.random.choice(["I just do.", "I was programmed to.", "I do Disney trivia on weekends",
                             "M A G I C"])
        print(r)
        print()
        return 1
        
    if ('im' in s_words):
        resp_start_i = s_words.index('im') + 1
        resp_list = s_words[resp_start_i:]
        for i in range(len(resp_list)):
            response = response + ' ' + resp_list[i]
        print(response)
        print()
        return 1
    
    if (('i' in s_words) and ('am' in s_words)):
        resp_start_i = s_words.index('am') + 1
        resp_list = s_words[resp_start_i:]
        for i in range(len(resp_list)):
            response = response + ' ' + resp_list[i]
        print(response)
        print()
        return 1
    
    if (('why' in s_words) and ('are' in s_words) and ('you' in s_words)):
        print("I don't know, why do you think?")
        print()
        return 1
    
    if s == 'yes':
        print('I see')
        print()
        return 1
    
    return 0

# Helper Functions

In [7]:
def lookup(name):
    """Function to return a description of a character or experiment.
    name should be a string
    Searches through the characters dictionary and the experiment table"""
    if name in list(exp['name_lower']):
        return exp[exp['name_lower']==name].iloc[0]['Short Description']
    elif name in characters.keys():
        return characters[name]
    elif 'experiment' in name.split():
        num = int(name.split()[1])
        return exp[exp['Num']==num].iloc[0]['Short Description']
    else:
        return "I'm sorry, I don't know who " + name + " is."

# One Variable Rules

In [8]:
# rule 1.1
def who_is(s):
    """One variable rule
    Answers questions about characters and experiments
    Ex: who is jumba"""
    m = re.search(r"who is (.+)", s)
    if (m):
        ans = lookup(m.groups()[0])
        print('ELIZA: ', ans)
        print()
        return 1
    return 0

In [9]:
# rule 1.2
def what_is_exp(s):
    """One variable rule
    Answers questions about the name of an experiment by number
    Ex: what is experiment 123"""
    m = re.search(r"what is experiment (.+)", s)
    if m:
        print('ELIZA: Experiment', m.groups()[0], 'is named', exp[exp['Num']==int(m.groups()[0])].iloc[0]['Name'])
        print()
        return 1
    return 0

In [10]:
# rule 1.3
def what_is_name(s):
    """One variable rule
    Answers questions in the format what is x when asking about an experiment by name.
    Ex: what is reuben"""
    m = re.search(r"what is (.+)", s)
    if m:
        match = m.groups()[0]
        if len(match.split()) > 1:
            return 0
        if match in list(exp['name_lower']):
            print('ELIZA: ', exp[exp['name_lower']==m.groups()[0]].iloc[0]['Short Description'])
            print()
            return 1
        elif match in characters.keys():
            print('ELIZA: ', characters[match])
            print()
            return 1
    return 0

In [11]:
# rule 1.4
def exp_description(s):
    """One variable rule
    Answers questions about what each experiment does when referred to by number
    Ex: what does experiment 123 do"""
    m = re.search(r"what does experiment (.+) do", s)
    if m:
        number = int(m.groups()[0])
        print('ELIZA: ', exp[exp['Num']==number].iloc[0]['Short Description'])
        print()
        return 1
    return 0

In [12]:
# rule 1.5
def exp_description_name(s):
    """One variable rule
    Answers questions about what each experiment does when referred to by name
    Ex: what does stitch do"""
    
    m = re.search(r"what does (.+) do", s)
    if m:
        name = m.groups()[0]
        if name in list(exp['name_lower']):
            print('ELIZA: ', exp[exp['name_lower']==name].iloc[0]['Short Description'])
            print()
            return 1
        elif name in characters.keys():
            print('ELIZA: ', characters[name])
            print()
            return 1
    return 0

In [13]:
# rule 1.6
def what_is_favorite_experiment(s):
    """One variable rule
    Answers questions about the bot's favorite experiment
    Ex: what is your favorite experiment"""
    m = re.search(r"what is (.+)", s)
    if m:
        words = m.groups()[0].split()
        if 'your' in words and 'favorite' in words:
            r = exp.sample()
            print('ELIZA: My favorite experiment is Experiment', r.iloc[0]['Num'],
                 'named', r.iloc[0]['Name'])
            print('What is your favorite experiment?')
            print()
            return 1
    return 0

In [14]:
# rule 1.7
def favorite_name(s):
    """One variable rule
    Responds to statement about the user's favorite character, when referred to by name
    Ex: my favorite is angel"""
    m = re.search(r"favorite is (.+)", s)
    
    if m:
        gr = m.groups()[0]
        name = gr.split()[0]
            
        if name in list(exp['name_lower']):
            print('ELIZA: Here is what I know about your favorite. ')
            print('Name: ', name)
            print('Experiment Number: ', exp[exp['name_lower']==name].iloc[0]['Num'])
            print(exp[exp['name_lower']==name].iloc[0]['Short Description'])
            print()
            return 1
        elif name in characters.keys():
            print('ELIZA: Here is what I know about your favorite. ', characters[name])
            print()
            return 1
    return 0

In [15]:
# rule 1.8
def favorite_experiment(s):
    """One variable rule
    Responds to statement about the user's favorite experiment, when referred to by number
    Ex: my favorite experiment is 234
    Ex: my favorite is experiment 234"""
    m = re.search(r"favorite experiment is (.+)", s)
    m1 = re.search(r"favorite is experiment (.+)", s)
    
    if m or m1:
        if m:
            if m.groups()[0] in list(exp['name_lower']):
                return favorite_name('favorite is ' + m.groups()[0]) 
            else:
                number = int(m.groups()[0])
        elif m1:
            number = int(m1.groups()[0])
        print('ELIZA: Here is what I know about experiment', number)
        print('Name: ', exp[exp['Num']==number].iloc[0]['Name'])
        print('Experiment Number: ', number)
        print(exp[exp['Num']==number].iloc[0]['Short Description'])
        print()
        return 1
    return 0

In [16]:
# rule 1.9
def homes(s):
    """One variable rule
    Responds to questions about where a character or experiment lives
    Ex: where does cobra live"""
    m = re.search(r"where does (.+) live", s)
    if m:
        match = m.groups()[0]
        if match in houses.keys():
            print('ELIZA: ', houses[match])
            print()
            return 1
        else:
            print("ELIZA: Sorry, I don't know where", match, "lives.")
            print()
            return 1
    return 0

In [17]:
# rule 1.10
def occupation(s):
    """One variable rule
    Responds to questions about the job each character has
    Ex: what is jumbas job"""
    m = re.search(r"what is (.+) job", s)
    if m:
        match = m.groups()[0]
        if match in jobs.keys():
            print('ELIZA: ', jobs[match])
            print()
            return 1
        else:
            print("ELIZA: Sorry, I don't know what", match, "job is.")
            print()
            return 1
    return 0

In [18]:
# bonus rule 1.11
def height(s):
    """One variable rule
    Responds to questions about the height of each character
    Ex: how tall is gantu"""
    
    m = re.search(r"how tall is (.+)", s)
    if m:
        match = m.groups()[0]
        if match in heights.keys():
            print('ELIZA: ', heights[match])
            print()
            return 1
        else:
            print("ELIZA: Sorry, I don't know how tall", match, 'is.')
            print()
            return 1
    return 0

# Two Variable Rules

In [19]:
# rule 2.1
def episodes(s):
    """Two variable rule
    Gives summaries of episodes of Lilo & Stitch: The Series 
    Ex: what happens in season 2 episode 14"""
    m = re.search(r"season (.+) episode (.+)", s)
    if m:
        season = int(m.groups()[0])
        episode = int(m.groups()[1])
        if season == 1:
            if episode > 39:
                print('There are only 39 episodes in season 1')
                return 1
            print('ELIZA: ', ep[(ep['season'] == season) & (ep['episode'] == episode)].iloc[0]['description'])
            print()
            return 1
        elif season == 2:
            if episode > 26:
                print('There are only 26 episodes in season 2')
                return 1
            print('ELIZA: ', ep[(ep['season'] == season) & (ep['episode'] == episode)].iloc[0]['description'])
            print()
            return 1
        else:
            print('There are only 2 seasons of Lilo & Stitch: The Series')
            print()
            return 1
    return 0

In [20]:
# rule 2.2
def better_than(s):
    """Two variable rule
    Responds to statements asserting that one thing is better than another
    Ex: lilo is better than stitch"""
    m = re.search(r"(.+) is better than (.+)", s)
    if m:
        item1 = m.groups()[0]
        item2 = m.groups()[1]
        print("I disagree. I think that", item2, "is better than", item1)
        print()
        lu = lookup(item2)
        if 'sorry' not in lu.split():
            print(lu)
            print()
        return 1
    return 0

In [21]:
# rule 2.3
def love_two(s):
    """Two variable rule
    Responds to a statement about loving two things.
    Ex: I love lilo and stitch."""
    m = re.search(r"love (.+) and (.+)", s)
    if m:
        item1 = m.groups()[0]
        item2 = m.groups()[1]
        print('I also love', item2, '. Tell me more about', item1)
        print()
        return 1
    return 0

In [22]:
# rule 2.4
def travel_to(s):
    """Two variable rule
    Responds to statements about characters traveling places or going places
    Ex: does lilo travel to space
    Ex: does lilo go to space"""
    m = re.search(r"does (.+) travel to (.+)", s)
    m1 = re.search(r"does (.+) go to (.+)", s)
    if m or m1:
        if m:
            name = m.groups()[0]
            loc = m.groups()[1]
        else: 
            name = m1.groups()[0]
            loc = m1.groups()[1]
        if name not in locations.keys():
            print('ELIZA: Sorry, I do not know where', name, 'travels to')
            print()
            return 1
        places = locations[name]
        if loc in places:
            print('ELIZA: Yes,', name, 'goes to', loc)
            print()
            return 1
        else:
            print('ELIZA: No,', name, 'does not go to', loc)
            print()
            return 1
    return 0        

In [23]:
# rule 2.5
def abilities(s):
    """Two variable rule
    Responds to questions about whether a character has an ability
    Ex: can stitch jump
    Ex: is it possible for stitch to jump
    """
    m = re.search(r"can (.+)", s)
    m1 = re.search(r"is it possible for (.+) to (.+)", s)
    
    if m or m1:
        if m:
            name = m.groups()[0].split()[0]
            skill = m.groups()[0].split()[1]
        else:
            name = m1.groups()[0]
            skill = m1.groups()[1]
        description = lookup(name)
        words = description.split()
        if 'sorry' in words:
            print("ELIZA: Sorry, I'm not sure if", name, 'can', skill)
            print()
            return 1
        elif skill in words:
            print('ELIZA: Yes,', name, 'can', skill)
            print('In fact,', name, 'can do many things.', description)
            print()
            return 1
        else:
            print("ELIZA: Sorry, I'm not sure if", name, 'can', skill)
            print()
            return 1
    return 0

# Interacting with the chatbot

In [24]:
def interact():
    """Function to interact with the chatbot."""
    print("Hi. Ask me about Lilo and Stitch!")
    
    while True:
        user = input('User >>> ')
        user = user.lower()
        user = re.sub('[\'.,!@#$]', '', user)
        a = 0 # keep track of whether question was truly answered
        if user in ['quit', 'stop', 'exit']:
            print('Goodbye')
            break
            
        # basic interaction
        a += hello(user)            
        a += common_questions(user)
        
        # one variable rules
        a += who_is(user) # only runs if user input matches the rule
        a += what_is_exp(user)
        a += what_is_name(user)
        a += exp_description(user)
        a += exp_description_name(user)
        a += what_is_favorite_experiment(user)
        a += favorite_name(user)
        a += favorite_experiment(user)
        a += homes(user)
        a += occupation(user)
        a += height(user)
        
        # two variable rules
        a += episodes(user)
        a += better_than(user)
        a += love_two(user)
        a += travel_to(user)
        a += abilities(user)
        
        
        # if no output has been printed
        # reframe the input as a question to user
        if a==0: 
            na = np.random.choice(non_answers) 
            na = na + user
            print('ELIZA:', na, '?')
            print('')